In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## Deep learning models

In [82]:
df_resale = pd.read_csv("../data/cleaned/reslae_price_normalized_for_ML.csv")
df_resale = pd.get_dummies(df_resale, columns=['town'], prefix='town')
df_resale['year'] = pd.to_datetime(df_resale['month']).dt.year
df_resale['month'] = pd.to_datetime(df_resale['month']).dt.month
df_resale.drop(columns=['flat_model','OtherReligions','building_age_2025','fx_rate','Chinese','Malays','Indians','Others'], inplace=True)
df_resale

,month,flat_type,storey_range,floor_area_sqm,remaining_lease,resale_price,lat,lon,nearest_mrt_distance,nearest_bus_distance,...,town_PUNGGOL,town_QUEENSTOWN,town_SEMBAWANG,town_SENGKANG,town_SERANGOON,town_TAMPINES,town_TOA PAYOH,town_WOODLANDS,town_YISHUN,year
0,3,0.333333,0.2500,0.083408,0.312121,0.131278,0.000000,0.456109,0.159033,0.226134,...,False,False,False,False,False,False,False,False,False,2017
1,5,0.333333,0.1250,0.083408,0.309346,0.131197,0.000000,0.456109,0.159033,0.226134,...,False,False,False,False,False,False,False,False,False,2017
2,3,0.333333,0.0625,0.083408,0.312121,0.127206,0.002888,0.454289,0.170487,0.124639,...,False,False,False,False,False,False,False,False,False,2017
3,12,0.333333,0.1875,0.116175,0.299116,0.171393,0.002888,0.454289,0.170487,0.124639,...,False,False,False,False,False,False,False,False,False,2017
4,1,0.333333,0.1250,0.101281,0.310733,0.190853,0.005513,0.415298,0.035276,0.242965,...,False,False,False,False,False,False,False,False,False,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196981,4,0.666667,0.0625,0.235329,0.663256,0.263095,1.000000,0.429908,0.288495,0.270891,...,False,False,True,False,False,False,False,False,False,2024
196982,7,0.666667,0.1250,0.235329,0.658921,0.277557,1.000000,0.429908,0.288495,0.270891,...,False,False,True,False,False,False,False,False,False,2024
196983,8,0.500000,0.2500,0.178731,0.657534,0.240401,1.000000,0.429908,0.288495,0.270891,...,False,False,True,False,False,False,False,False,False,2024
196984,10,0.666667,0.2500,0.238308,0.654586,0.329079,1.000000,0.429908,0.288495,0.270891,...,False,False,True,False,False,False,False,False,False,2024


In [83]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separate features and target
X = df_resale.drop(columns=['resale_price'])
y = df_resale['resale_price']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=False
)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the MLR model
mlr_model = LinearRegression()
mlr_model.fit(X_train_scaled, y_train)

# Make predictions
mlr_predictions = mlr_model.predict(X_test_scaled)
mlr_r2, mlr_rmse, mlr_mae = evaluate_model(y_test, mlr_predictions)

print(f"MLR - R²: {mlr_r2:.4f}, RMSE: {mlr_rmse:.4f}, MAE: {mlr_mae:.4f}")


MLR - R²: 0.7103, RMSE: 0.0513, MAE: 0.0409


## Preprocessing

In [ ]:
X = df_resale.drop(columns=['resale_price'])
y = df_resale['resale_price']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

## Model Definition

### Simple Neural Network

In [24]:
nn_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

nn_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
nn_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

nn_predictions = nn_model.predict(X_test)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1232/1232 ━━━━━━━━━━━━━━━━━━━━ 0s 224us/step


### RNN

In [26]:
# Reshape input to 3D: (samples, timesteps, features)
X_train_rnn = X_train.to_numpy().astype(np.float32).reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = X_test.to_numpy().astype(np.float32).reshape((X_test.shape[0], 1, X_test.shape[1]))
y_train_rnn = y_train.astype(np.float32)

# Define RNN model
rnn_model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(64, activation='tanh', input_shape=(1, X_train.shape[1])),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

rnn_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

rnn_model.fit(X_train_rnn, y_train_rnn, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

rnn_predictions = rnn_model.predict(X_test_rnn)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1232/1232 ━━━━━━━━━━━━━━━━━━━━ 0s 275us/step


### LSTM

In [ ]:
new_lstm_model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, activation='tanh', return_sequences=True, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    tf.keras.layers.LSTM(32, activation='tanh'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)
])

new_lstm_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

new_early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

new_lstm_model.fit(X_train_rnn, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1, callbacks=[new_early_stopping])
# Predictions
lstm_predictions = new_lstm_model.predict(X_test_rnn)

Epoch 1/50
3940/3940 ━━━━━━━━━━━━━━━━━━━━ 5s 862us/step - loss: 0.0247 - mae: 0.1245 - val_loss: 0.0069 - val_mae: 0.0657
Epoch 2/50
3940/3940 ━━━━━━━━━━━━━━━━━━━━ 3s 827us/step - loss: 0.0211 - mae: 0.1171 - val_loss: 0.0068 - val_mae: 0.0646
Epoch 3/50
3940/3940 ━━━━━━━━━━━━━━━━━━━━ 3s 834us/step - loss: 0.0212 - mae: 0.1174 - val_loss: 0.0070 - val_mae: 0.0668
Epoch 4/50
3940/3940 ━━━━━━━━━━━━━━━━━━━━ 3s 843us/step - loss: 0.0212 - mae: 0.1177 - val_loss: 0.0068 - val_mae: 0.0653
Epoch 5/50
3940/3940 ━━━━━━━━━━━━━━━━━━━━ 3s 837us/step - loss: 0.0213 - mae: 0.1179 - val_loss: 0.0071 - val_mae: 0.0677
Epoch 6/50
3940/3940 ━━━━━━━━━━━━━━━━━━━━ 3s 841us/step - loss: 0.0212 - mae: 0.1179 - val_loss: 0.0068 - val_mae: 0.0648
Epoch 7/50
3940/3940 ━━━━━━━━━━━━━━━━━━━━ 3s 836us/step - loss: 0.0212 - mae: 0.1177 - val_loss: 0.0071 - val_mae: 0.0673


## Evaluation

In [84]:
def evaluate_model(true_values, predictions):
    r2 = r2_score(true_values, predictions)
    rmse = np.sqrt(mean_squared_error(true_values, predictions))
    mae = mean_absolute_error(true_values, predictions)
    return r2, rmse, mae